In [0]:
%pip install typing_extensions==3.7.4.3
%pip install torch==2.0.0
%pip install torch-geometric
!pip install folium


In [ ]:
from pyspark.sql import SparkSession
import glob
from pyspark.sql.functions import col, dayofmonth, to_timestamp, date_format, explode, sequence, expr, coalesce, when, to_date, lit, substring, concat, hour, avg
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, TimestampType
from pyspark.ml.feature import StringIndexer
from pyspark import StorageLevel
import folium
from folium.plugins import MarkerCluster, HeatMap
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [0]:
# Se crea una sesión de Spark
spark = SparkSession.builder.appName("UberDataProcessing").getOrCreate()

# Se definen las rutas de los archivos (en este script los datos de tiempo y eventos se importaron de unos fihceros previamnete descargados)
taxi_zones_path = "/FileStore/tables/taxi_zones_with_coordinates.csv"
taxi_zones_lookup_path = "/FileStore/tables/taxi_zone_lookup.csv"
weather_df_path = "/FileStore/tables/weather_nyc_2024.csv"
events_data_path = "/FileStore/tables/events.csv"

# Carga de los archivos CSV infiriendo el esquema
taxi_zones_df = spark.read.csv(taxi_zones_path, header=True, inferSchema=True)
taxi_zones_lookup_df = spark.read.csv(taxi_zones_lookup_path, header=True, inferSchema=True)
weather_df = spark.read.csv(weather_df_path, header=True, inferSchema=True)
events_df = spark.read.option("sep", ";").csv(events_data_path, header=True, inferSchema=True)

# Lista de archivos a cargar
file_paths = [
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_01.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_02.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_03.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_04.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_05.parquet",
    "dbfs:/FileStore/tables/fhvhv_tripdata_2024_06.parquet",
]

# Se cargan los archivos Parquet
df_list = []
for file_path in file_paths:
    temp_df = spark.read.parquet(file_path)
    df_list.append(temp_df)

# Se concatenan todos los DataFrames de la lista
uber_df = df_list[0]
for df in df_list[1:]:
    uber_df = uber_df.union(df)

uber_df = uber_df.filter((dayofmonth(col("request_datetime")) >= 1) & (dayofmonth(col("request_datetime")) <= 31))

# Mostrar los esquemas que se han inferido
taxi_zones_df.printSchema()
taxi_zones_lookup_df.printSchema()
uber_df.printSchema()

# Imprimo en pantalla la cantidad de filas en el DataFrame concatenado
print(f"Cantidad de filas en el DataFrame concatenado: {uber_df.count()}")


In [0]:
# Se elige una muestra manejable 
uber_df = uber_df.sample(fraction=0.025, seed=42)
# Imprimo en pantalla la cantidad de filas en el DataFrame concatenado
print(f"Cantidad de filas en el DataFrame concatenado: {uber_df.count()}")

In [0]:


# Verifico  cuántas veces aparece 'date' en las columnas
date_columns = [c for c in uber_df.columns if 'date' in c.lower()]  # Identificamos columnas con 'date' en su nombre
print(f"Columnas que contienen 'date': {date_columns}")


# Elimino columnas duplicadas
for i, col in enumerate(date_columns):
    new_name = f"{col}_{i+1}" if date_columns.count(col) > 1 else col
    uber_df = uber_df.withColumnRenamed(col, new_name)

# Identifico columnas numéricas y no numéricas
numeric_cols = [
    c for c in uber_df.columns 
    if uber_df.select(c).schema.fields[0].dataType.typeName() in ['integer', 'float', 'double']
]
print(f"Columnas numéricas: {numeric_cols}")

# Identifico las columnas no numéricas
non_numeric_cols = [c for c in uber_df.columns if c not in numeric_cols]
print(f"Columnas no numéricas: {non_numeric_cols}")

# Calculo valores faltantes

missing_values = uber_df.select([
    (F.when(uber_df[c].isNull(), 1).otherwise(0)).alias(c) for c in uber_df.columns
])

# Uso la función 'sum' de PySpark para calcular los valores faltantes por columna
missing_values_summary = missing_values.agg(*[
    F.sum(missing_values[c]).alias(c) for c in uber_df.columns
]).collect()

# Imprimo en pantalla el resumen de valores faltantes
print(f"Resumen de valores faltantes: {missing_values_summary}")


In [0]:
display(missing_values_summary)

In [0]:

# Creo lel filtro de las zonas de Manhattan en el df de las zonas de Taxi
manhattan_location_ids = taxi_zones_df.filter(col("borough") == "Manhattan") \
                                       .select("LocationID") \
                                       .rdd.flatMap(lambda x: x).collect()

# Filtro los datos de hide-railing para las rutas que son completamente dentro de Manhattan
uber_df = uber_df.filter(
    (col("PULocationID").isin(manhattan_location_ids) & col("DOLocationID").isin(manhattan_location_ids))
)


In [0]:
print(f"Cantidad de filas en el DataFrame concatenado: {uber_df.count()}")

In [0]:
uber_df.show()

In [0]:
events_df.show()

In [0]:
# Configuro la política  de fechas
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Elimino duplicados
events_df = events_df.dropDuplicates()

# Uniformizo las columnas de fecha a tipo Timestamp, manejando ambos formatos de fecha
events_df = events_df.withColumn(
    "Start Date/Time",
    coalesce(
        to_timestamp(col("Start Date/Time"), "dd/MM/yyyy HH:mm"),
        to_timestamp(col("Start Date/Time"), "MM/dd/yyyy hh:mm:ss a")
    )
).withColumn(
    "End Date/Time",
    coalesce(
        to_timestamp(col("End Date/Time"), "dd/MM/yyyy HH:mm"),
        to_timestamp(col("End Date/Time"), "MM/dd/yyyy hh:mm:ss a")
    )
)

# Filtro las filas donde End Date/Time es anterior a Start Date/Time
events_df = events_df.filter(col("End Date/Time") >= col("Start Date/Time"))

# Creo la secuencia de horas entre Start Date/Time y End Date/Time usando INTERVAL 1 HOUR
events_with_hours_df = events_df.withColumn(
    "hour_sequence",
    explode(sequence(
        col("Start Date/Time"),
        col("End Date/Time"),
        expr("INTERVAL 1 HOUR")
    ))
)

# Extraigo la fecha y la hora de la secuencia generada
events_with_hours_df = events_with_hours_df.withColumn("Date", date_format(col("hour_sequence"), "yyyy-MM-dd")) \
                                           .withColumn("Hour", date_format(col("hour_sequence"), "HH:00"))

# Creo una columna que marque si un evento ocurre en esa hora
events_with_hours_df = events_with_hours_df.withColumn(
    "Event Count", 
    when(col("Event Type").isNotNull(), 1).otherwise(0)
)

# Agrupo por fecha, hora y tipo de evento, luego pivoto para contar los eventos por tipo de evento
aggregated_events_df = events_with_hours_df.groupBy("Date", "Hour").pivot("Event Type").agg({"Event Count": "sum"})

# Relleno valores nulos con 0
aggregated_events_df = aggregated_events_df.fillna(0)
# Se suman todas las columnas que no son ni 'Date' ni 'Hour' para obtener la columna 'nº events'
aggregated_events_df = aggregated_events_df.withColumn(
    "nº events",
    sum([F.col(col).cast("int") for col in aggregated_events_df.columns if col not in ["Date", "Hour"]])
)
# Imprimo en pantalla el resultado
aggregated_events_df.show(10, truncate=False)


In [0]:
display(events_with_hours_df)

In [0]:
display(aggregated_events_df)

In [0]:

display(uber_df)

In [0]:
# Selecciono las columnas requeridas de taxi_zones_df
taxi_zones_df_filtered = taxi_zones_df.select("LocationID", "borough", "lat", "lon")

# Uno uber_df con taxi_zones_df_filtered en la columna "PULocationID" (id de recogida)
uber_df = uber_df.join(
    taxi_zones_df_filtered,
    uber_df["PULocationID"] == taxi_zones_df_filtered["LocationID"],
    how="left"
).drop("LocationID")  # Elimino el LocationID duplicado tras la unión

# Renombro las columnas lat y lon como "pickup_lat" y "pickup_lon" para evitar futuros duplicados con las coodenadas de recogida 
uber_df = uber_df.withColumnRenamed("lat", "pickup_lat").withColumnRenamed("lon", "pickup_lon")

# Se imprime en pantalla el resultado final
uber_df.show()


In [0]:
taxi_zones_df_filtered.show()

In [0]:
taxi_zones_df.show()

In [0]:
# Renombro las columnas en taxi_zones_df_filtered antes de la unión (en este caso hago lo propio antes para traerme las coordenadas de destino)
taxi_zones_df_filtered = taxi_zones_df.select("LocationID", "lat", "lon") \
    .withColumnRenamed("lat", "dropoff_lat") \
    .withColumnRenamed("lon", "dropoff_lon")

# Asigno alias a los DataFrames antes de la unión (tuve problemas para unirlos con los mismos nombres)
uber_df_alias = uber_df.alias("uber")
taxi_zones_df_filtered_alias = taxi_zones_df_filtered.alias("taxi_zones")

# Realizo la unión usando los alias y eliminando columnas redundantes
uber_df = uber_df_alias.join(
    taxi_zones_df_filtered_alias,
    uber_df_alias["DOLocationID"] == taxi_zones_df_filtered_alias["LocationID"],
    how="left"
).drop(taxi_zones_df_filtered_alias["LocationID"])  # Elimino de nuevo el LocationID redundante

# Imprimo en pantalla el resultado final
uber_df.show()


In [0]:
uber_df.printSchema()

In [0]:
# Limpio el DataFrame de hide-railing y se calculan también las columnas requeridas
uber_cleaned = uber_df.select(
    # Coordenadas de salida (latitud y longitud)
    F.col("pickup_lat").alias("pickup_latitude"),
    F.col("pickup_lon").alias("pickup_longitude"),
    # ID de localización
    F.col("PULocationID").alias("pickup_location_id"),
    F.col("DOLocationID").alias("dropoff_location_id"),

    # Coordenadas de llegada (latitud y longitud)
    F.col("dropoff_lat").alias("dropoff_latitude"),
    F.col("dropoff_lon").alias("dropoff_longitude"),

    # Fecha y hora de la solicitud
    F.col("request_datetime").alias("request_datetime"),

    # Precio total sumando las tarifas y tasas, utilizando coalesce para manejar posibles NULLs
    (
        F.coalesce(F.col("base_passenger_fare"), F.lit(0)) +
        F.coalesce(F.col("tolls"), F.lit(0)) +
        F.coalesce(F.col("bcf"), F.lit(0)) +
        F.coalesce(F.col("sales_tax"), F.lit(0)) +
        F.coalesce(F.col("congestion_surcharge"), F.lit(0)) +
        F.coalesce(F.col("airport_fee"), F.lit(0))
    ).alias("total_price"),

    # Retraso en minutos (calculado como la diferencia entre pickup_datetime y request_datetime)
    ((F.unix_timestamp(F.col("pickup_datetime")) - F.unix_timestamp(F.col("request_datetime"))) / 60).alias("delay_minutes"),

    # Tiempo total del viaje en minutos (calculado como la diferencia entre dropoff_datetime y request_datetime)
    ((F.unix_timestamp(F.col("dropoff_datetime")) - F.unix_timestamp(F.col("pickup_datetime"))) / 60).alias("total_trip_time_minutes"),

    # Licencia
    F.col("hvfhs_license_num").alias("license"),

    # Conversión de millas a kilómetros (en el trabajo hablo siempre de kilómetros)
    (F.col("trip_miles") * 1.60934).alias("trip_kilometers"),
    # Hora del día extraída de request_datetime
    F.hour(F.col("request_datetime")).alias("hour_of_day"),

    # Día de la semana extraído de request_datetime (1 = Domingo, 7 = Sábado)
    F.dayofweek(F.col("request_datetime")).alias("day_of_week"),
    F.to_date(F.col("request_datetime")).alias("date")
)
#Elimino valores faltantes de dropoff longitud ya que hay dos id de zonas que no se disponen, por lo que se ha  decidido eliminar
uber_cleaned = uber_cleaned.dropna(subset=["dropoff_longitude"])




In [0]:
#  Imprimo la cantidad de filas del df para comprobar si ha sucedido algo raro
print(f"Cantidad de filas en el DataFrame concatenado: {uber_cleaned.count()}")

In [0]:
display(uber_cleaned)


In [0]:
# Rango de fechas
fecha_min = "2024-01-01"
fecha_max = "2024-07-01"

# Me cercioro de que las fechas estén en formato adecuado
uber_cleaned = uber_cleaned.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
aggregated_events_df = aggregated_events_df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

# Filtro por rango de fechas
uber_cleaned = uber_cleaned.filter((col("date") >= lit(fecha_min)) & (col("date") <= lit(fecha_max)))
aggregated_events_df = aggregated_events_df.filter((col("Date") >= lit(fecha_min)) & (col("Date") <= lit(fecha_max)))

# Aquí ajusto el formato de la hora para que sean equivalentes

# Convierto `hour_of_day` (entero) a formato "14:00"
uber_cleaned = uber_cleaned.withColumn("hour_of_day_formatted", concat(col("hour_of_day").cast("string"), lit(":00")))

# Convierto `Hour` (en formato "14:00") a tipo  entero "14"
aggregated_events_df = aggregated_events_df.withColumn("Hour_int", substring(col("Hour"), 1, 2).cast("int"))

# Renombro columnas con espacios o caracteres especiales
aggregated_events_df = aggregated_events_df.withColumnRenamed("nº events", "num_events")
print(f"Registros antes del join: {uber_cleaned.count()}")
# Hago un left join entre uber_cleaned y aggregated_events_df
uber_cleaned = uber_cleaned.join(
    aggregated_events_df,
    (uber_cleaned.date == aggregated_events_df.Date) & (uber_cleaned.hour_of_day == aggregated_events_df.Hour_int),
    how="left"
)
print(f"Registros después del join: {uber_cleaned.count()}")

# Relleno los valores null en la columna 'num_events' con 0
uber_cleaned = uber_cleaned.fillna({"num_events": 0})

# Renombro de vuelta la columna `num_events` a `nº events`
uber_cleaned = uber_cleaned.withColumnRenamed("num_events", "nº events")

# Imprimo el df combinado
display(uber_cleaned)


In [0]:

weather_df = weather_df.withColumnRenamed("index", "time")

# Me aseguro  de que el tipo de dato es timestamp
weather_df = weather_df.withColumn("time", F.col("time").cast("timestamp"))
weather_df.fillna(0)



In [0]:

# esto es para dinamizar la columna coco
# Paso a string la columna coco
weather_df = weather_df.withColumn("coco", F.col("coco").cast("string"))

# Extraigo los valores únicos de la columna 'coco' para crear una columna por cada valor
unique_values = weather_df.select("coco").distinct().rdd.flatMap(lambda x: x).collect()

# Creo dinámicamente una columna para cada valor único en 'coco'
for value in unique_values:
    weather_df = weather_df.withColumn(
        f"coco_{value}",
        F.when(F.col("coco") == value, 1).otherwise(0)
    )

In [0]:
display(weather_df)

In [0]:
# Realizo la unión, cruzando los datos por fecha y hora
uber_weather_df = uber_cleaned.join(
    weather_df,
    (uber_cleaned["request_datetime"] >= weather_df["time"]) & (uber_cleaned["request_datetime"] < (weather_df["time"] + F.expr("INTERVAL 1 HOUR"))),
    how="left"
)

In [0]:
uber_weather_df.show()

In [0]:
# Obtengo la cantidad de columnas
num_columnas = len(uber_weather_df.columns)

# Realizo un muestreo para obtener una aproximación del conteo de filas, mejor que .count() en datos grandes
num_filas = uber_weather_df.rdd.countApprox(timeout=5000)  # Ajusta el timeout según el tamaño de tu dataset

print(f"El DataFrame tiene aproximadamente {num_filas} filas y {num_columnas} columnas.")


In [0]:
# Filtro las  filas donde las tres columnas tienen valores mayores o iguales a 0 (se considera imposible y un error)
uber_weather_df = uber_weather_df[
    (uber_weather_df['total_price'] >= 0) &
    (uber_weather_df['delay_minutes'] >= 0) &
    (uber_weather_df['total_trip_time_minutes'] >= 0)
]



In [0]:
# Calculo el tiempo total de viaje en horas
uber_weather_df = uber_weather_df.withColumn(
    'total_trip_time_hours', 
    uber_weather_df['total_trip_time_minutes'] / 60
)

# Calculo la velocidad promedio en km/h
uber_weather_df = uber_weather_df.withColumn(
    'speed_kmh', 
    uber_weather_df['trip_kilometers'] / uber_weather_df['total_trip_time_hours']
)


In [0]:
print(uber_weather_df.columns)


In [0]:
# Cuento los valores nulos en la columna 'total_price'

null_count = uber_weather_df.filter(uber_weather_df.total_price.isNull()).count()
print(f"Cantidad de valores nulos en 'total_price': {null_count}")


In [0]:
uber_weather_df.cache()
# Selecciono de columnas de duración y distancia para descripción valores estadísticos generales 
uber_weather_df.select("total_trip_time_minutes", "trip_kilometers","total_price","delay_minutes", "speed_kmh").describe().show()


In [0]:
uber_weather_df.cache()
# Resumen de temperatura y humedad
uber_weather_df.select("temp", "rhum").describe().show()


In [0]:
uber_weather_df.cache()
# Relación entre precio total y temperatura
uber_weather_df.groupBy("temp").avg("total_price").show()


In [0]:
uber_weather_df.cache()
# Ubicaciones de recogida más frecuentes
uber_weather_df.groupBy("pickup_location_id").count().orderBy("count", ascending=False).show(10)
# Ubicaciones de destino más frecuentes
uber_weather_df.groupBy("dropoff_location_id").count().orderBy("count", ascending=False).show(10)


In [0]:
uber_weather_df.cache()
# Promedio de precio total por hora del día
uber_weather_df.withColumn("hour", hour("request_datetime")) \
               .groupBy("hour").avg("total_price").orderBy("hour").show()


In [0]:
# Configuro el estilo de seaborn
sns.set(style="whitegrid")

# Convierto DataFrame de PySpark a pandas
uber_weather_pd_df = uber_weather_df.toPandas()

# Selecciono las columnas numéricas para análisis de correlación
numeric_columns = uber_weather_pd_df.select_dtypes(include=['number']).columns
uber_weather_numeric_df = uber_weather_pd_df[numeric_columns]  # DataFrame solo con columnas numéricas

# Creo una figura con varios subplots
fig, axes = plt.subplots(3, 2, figsize=(15, 18))  # Aumentamos el tamaño para más subplots

# 1. Histograma de la distribución de los precios
sns.histplot(uber_weather_pd_df['total_price'], bins=30, kde=True, color="skyblue", ax=axes[0, 0])
axes[0, 0].set_title("Distribución de los Precios Totales")
axes[0, 0].set_xlabel("Precio Total")
axes[0, 0].set_ylabel("Frecuencia")

# 2. Dispersión de retrasos vs. tiempo total de viaje
sns.scatterplot(data=uber_weather_pd_df, x="total_trip_time_minutes", y="delay_minutes", ax=axes[0, 1], color="salmon")
axes[0, 1].set_title("Retrasos en función del Tiempo Total del Viaje")
axes[0, 1].set_xlabel("Tiempo Total del Viaje (minutos)")
axes[0, 1].set_ylabel("Retraso (minutos)")

# 3. Dispersión de precio vs. tiempo total de viaje
sns.scatterplot(data=uber_weather_pd_df, x="total_trip_time_minutes", y="total_price", ax=axes[1, 0], color="lightgreen")
axes[1, 0].set_title("Relación entre Precio y Tiempo Total del Viaje")
axes[1, 0].set_xlabel("Tiempo Total del Viaje (minutos)")
axes[1, 0].set_ylabel("Precio Total")

# 4. Boxplots para detectar valores extremos
sns.boxplot(data=uber_weather_pd_df[['total_price', 'delay_minutes', 'total_trip_time_minutes']], ax=axes[1, 1])
axes[1, 1].set_title("Boxplots de Precios, Retrasos y Tiempo Total de Viaje")
axes[1, 1].set_ylabel("Valor")

# 5. Subplot de la velocidad promedio por hora

# Calculo primero la velocidad promedio por hora
speed_by_hour = uber_weather_pd_df.groupby('hour_of_day')['speed_kmh'].mean().reset_index()
# Genero como tal el gráfico
sns.lineplot(data=speed_by_hour, x='hour_of_day', y='speed_kmh', marker='o', ax=axes[2, 0], color="purple")
axes[2, 0].set_title("Velocidad Promedio por Hora")
axes[2, 0].set_xlabel("Hora del Día")
axes[2, 0].set_ylabel("Velocidad Promedio (km/h)")

# 6. Subplot de un mapa de calor de los pares de pickup_location_id y dropoff_location_id
# Creo una tabla de frecuencias de los pares de ubicaciones
pickup_dropoff_counts = uber_weather_pd_df.groupby(['pickup_location_id', 'dropoff_location_id']).size().reset_index(name='count')

# Pivoto la tabla para crear la matriz de correlación
heatmap_data = pickup_dropoff_counts.pivot('pickup_location_id', 'dropoff_location_id', 'count').fillna(0)

sns.heatmap(heatmap_data, cmap="YlGnBu", ax=axes[2, 1], cbar_kws={'label': 'Frecuencia'}, annot=False)
axes[2, 1].set_title("Mapa de Calor: Pares Recogida vs Destino más Demandados")
axes[2, 1].set_xlabel("ID Destino")
axes[2, 1].set_ylabel("ID Recogida")

# Ajusto el layout para evitar solapamientos
plt.tight_layout()

# Imprimo los gráficos en pantalla
plt.show()





In [0]:
uber_weather_df.cache()
# Distribución de la velocidad de circulación
uber_weather_df.select("speed_kmh").describe().show()


In [0]:

# Quito los valores atípicos: la lógica se define en el documento
# Lo dejo el df en memoria para acelerar operaciones futuras
uber_weather_df = uber_weather_df.persist(StorageLevel.MEMORY_AND_DISK)

# Calculo los cuartiles e IQR solo para la columna de precio
Q1_price, Q3_price = uber_weather_df.approxQuantile("total_price", [0.25, 0.75], 0.05)
IQR_price = Q3_price - Q1_price

# Filtro valores atípicos y establecer límites lógicos
uber_weather_df = uber_weather_df.filter(
    (col("trip_kilometers") <= 30) &  # Distancia máxima lógica para Manhattan
    (col("speed_kmh") <= 80) &         # Velocidad máxima permitida
    (col("total_trip_time_minutes") <= 120) &  # Tiempo total máximo permitido
    (col("total_price") >= (Q1_price - 1.5 * IQR_price)) &  # Precio dentro del rango intercuartílico extendido
    (col("total_price") <= 200) &    # Máximo precio permitido es 200 USD
    (col("delay_minutes") <= 120) &  # Máximo retraso permitido
    (col("total_trip_time_minutes") >= 0)  # Tiempo de viaje no negativo
)

#  Ajusto particiones para mejorar el rendimiento , no es necesario pero mejora el rendimiento
uber_weather_df = uber_weather_df.repartition("total_price")

# Muestro resultados limitados solo 5 
uber_weather_df.show(5, truncate=False)

# Libero memoria
uber_weather_df.unpersist()


In [0]:
# Configuro el estilo de seaborn
sns.set(style="whitegrid")

# Convierto el df de PySpark a pandas
uber_weather_pd_df = uber_weather_df.toPandas()

# Selecciono las columnas numéricas para análisis de correlación
numeric_columns = uber_weather_pd_df.select_dtypes(include=['number']).columns
uber_weather_numeric_df = uber_weather_pd_df[numeric_columns]  # DataFrame solo con columnas numéricas

# Creo una figura con múltiples subplots
fig, axes = plt.subplots(3, 2, figsize=(15, 18))  # Aumentamos el tamaño para incluir más subplots

# 1. Histograma de la distribución de los precios
sns.histplot(uber_weather_pd_df['total_price'], bins=30, kde=True, color="skyblue", ax=axes[0, 0])
axes[0, 0].set_title("Distribución de los Precios Totales")
axes[0, 0].set_xlabel("Precio Total")
axes[0, 0].set_ylabel("Frecuencia")

# 2. Dispersión de retrasos vs. tiempo total de viaje
sns.scatterplot(data=uber_weather_pd_df, x="total_trip_time_minutes", y="delay_minutes", ax=axes[0, 1], color="salmon")
axes[0, 1].set_title("Retrasos en función del Tiempo Total del Viaje")
axes[0, 1].set_xlabel("Tiempo Total del Viaje (minutos)")
axes[0, 1].set_ylabel("Retraso (minutos)")

# 3. Dispersión de precio vs. tiempo total de viaje
sns.scatterplot(data=uber_weather_pd_df, x="total_trip_time_minutes", y="total_price", ax=axes[1, 0], color="lightgreen")
axes[1, 0].set_title("Relación entre Precio y Tiempo Total del Viaje")
axes[1, 0].set_xlabel("Tiempo Total del Viaje (minutos)")
axes[1, 0].set_ylabel("Precio Total")

# 4. Boxplots para detectar valores extremos
sns.boxplot(data=uber_weather_pd_df[['total_price', 'delay_minutes', 'total_trip_time_minutes']], ax=axes[1, 1])
axes[1, 1].set_title("Boxplots de Precios, Retrasos y Tiempo Total de Viaje")
axes[1, 1].set_ylabel("Valor")

# 5. Subplot de la velocidad promedio por hora

# Calculo la velocidad promedio por hora y el conteo (demanda)
speed_and_demand_by_hour = uber_weather_pd_df.groupby('hour_of_day').agg({
    'speed_kmh': 'mean',
    'hour_of_day': 'count'  # Esto cuenta las filas para representar la demanda
}).rename(columns={'hour_of_day': 'demand'}).reset_index()

sns.lineplot(
    data=speed_and_demand_by_hour, 
    x="hour_of_day", 
    y="demand", 
    marker="o", 
    ax=axes[2, 0], 
    color="blue", 
    label="Demanda",
    ci=95
)

# Configuro títulos y etiquetas
axes[2, 0].set_title("Demanda por Hora de Hide-Railing")
axes[2, 0].set_xlabel("Hora del Día")
axes[2, 0].set_ylabel("Demanda Total")
axes[2, 0].legend(title="Métrica")


# 6. Subplot de un mapa de calor de los pares de pickup_location_id y dropoff_location_id
# Creo una tabla de frecuencias de los pares de ubicaciones
pickup_dropoff_counts = uber_weather_pd_df.groupby(['pickup_location_id', 'dropoff_location_id']).size().reset_index(name='count')

# Pivoto la tabla para crear la matriz de correlación
heatmap_data = pickup_dropoff_counts.pivot('pickup_location_id', 'dropoff_location_id', 'count').fillna(0)

sns.heatmap(heatmap_data, cmap="YlGnBu", ax=axes[2, 1], cbar_kws={'label': 'Frecuencia'}, annot=False)

axes[2, 1].set_title("Mapa de Calor: Pares Recogida vs Destino más Demandados")
axes[2, 1].set_xlabel("ID Destino")
axes[2, 1].set_ylabel("ID Recogida")

# Ajusto el layout para evitar solapamientos
plt.tight_layout()

# Imprimo los gráficos en pantalla
plt.show()





In [ ]:
speed_and_demand_by_hour = uber_weather_pd_df.groupby('hour_of_day').agg({
    'speed_kmh': 'mean',
    'hour_of_day': 'count'  # Esto cuenta las filas para representar la demanda
}).rename(columns={'hour_of_day': 'demand'}).reset_index()

# Creo el escalador MinMax
scaler = MinMaxScaler()

# Normalizo las columnas "speed_kmh" y "demand"
speed_and_demand_by_hour[["speed_kmh", "demand"]] = scaler.fit_transform(
    speed_and_demand_by_hour[["speed_kmh", "demand"]]
)

# Creo el gráfico con las métricas normalizadas
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=speed_and_demand_by_hour, 
    x="hour_of_day", 
    y="speed_kmh", 
    marker="o", 
    ax=axes[2, 0], 
    color="purple", 
    label="Velocidad Promedio (Normalizada)"
)
sns.lineplot(
    data=speed_and_demand_by_hour, 
    x="hour_of_day", 
    y="demand", 
    marker="x", 
    ax=axes[2, 0], 
    color="orange", 
    label="Demanda (Normalizada)"
)
# Detalles del gráfico
plt.title("Velocidad Promedio y Demanda por Día de la Semana (Normalizadas)")
plt.xlabel("Día de la Semana")
plt.ylabel("Valores Normalizados")
plt.legend()
plt.grid()
plt.tight_layout()

# Muestro el gráfico
plt.show()

In [0]:
print(speed_and_demand_by_hour)

In [0]:

# Reparticiono el df antes de realizar la operación, esto ayuda a distribuir los datos de manera eficiente
uber_weather_df = uber_weather_df.repartition(100) 

# lo cacheo para optimizar el rendimiento
uber_weather_df.cache()

# Agrupo de manera específica (por ruta)
time_by_location_hour = (
    uber_weather_df
    .groupBy(
        F.col("pickup_latitude").alias("pickup_latitude"),
        F.col("pickup_longitude").alias("pickup_longitude"),
        F.col("dropoff_latitude").alias("dropoff_latitude"),
        F.col("dropoff_longitude").alias("dropoff_longitude"),
        F.to_date("request_datetime").alias("date"),
        F.hour("request_datetime").alias("hour"),
        F.dayofweek("request_datetime").alias("day_of_week")  # Añado el día de la semana
    )
    .agg(
        F.avg("total_trip_time_minutes").alias("avg_trip_time_minutes"),
        F.avg("temp").alias("avg_temp"),
        F.avg("trip_kilometers").alias("avg_distance"),
        F.avg("rhum").alias("avg_rhum"),
        F.avg("speed_kmh").alias("avg_speed_kmh"),
        F.avg("delay_minutes").alias("avg_delay_minutes"),
        F.avg("prcp").alias("avg_prcp"),
        F.avg("nº events").alias("avg_events"),
        F.count("*").alias("demand_count")
    )
)




In [0]:
display(time_by_location_hour)

In [0]:
# Agrupo de forma mas general para identificar mejor las correlaciones
time_by_location_hour_day = (
    uber_weather_df
    .groupBy(
        F.hour("request_datetime").alias("hour"),
        F.dayofweek("request_datetime").alias("day_of_week")  # Añado el día de la semana
    )
    .agg(
        F.avg("total_trip_time_minutes").alias("avg_trip_time_minutes"),
        F.avg("temp").alias("avg_temp"),
        F.avg("trip_kilometers").alias("avg_distance"),
        F.avg("rhum").alias("avg_rhum"),
        F.avg("wspd").alias("avg_wspd"),
        F.avg("speed_kmh").alias("avg_speed_kmh"),
        F.avg("delay_minutes").alias("avg_delay_minutes"),
        F.avg("prcp").alias("avg_prcp"),
        F.avg("nº events").alias("avg_events"),
        F.count("*").alias("demand_count")
    )
)


In [0]:
time_by_location_hour_day = time_by_location_hour_day.cache()

# Lo paso a pandas

time_by_location_hour_day_pd = time_by_location_hour_day.toPandas()

# Creo la matriz de correlación en Pandas para visualizarla con seaborn/matplotlib

correlation_matrix = time_by_location_hour_day_pd[["avg_trip_time_minutes", "avg_temp", "avg_rhum", "avg_speed_kmh", "avg_prcp", "avg_events", "avg_wspd", "demand_count","avg_distance", "avg_delay_minutes"]].corr()

# Configuro el tamaño del gráfico
plt.figure(figsize=(10, 8))

# Dibujo el mapa de calor
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, square=True, fmt=".2f",
            cbar_kws={"shrink": .8}, annot_kws={"size": 10})

# Configuración adicional del mapa de calor
plt.title("Matriz de Correlación General", fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.tight_layout()

# Imprimo en pantalla el gráfico
plt.show()

In [0]:
time_by_location_hour = time_by_location_hour.cache()

# Lo paso a pandas

time_by_location_hour_pd = time_by_location_hour.toPandas()

# Creo la matriz de correlación en Pandas para visualizarla con seaborn/matplotlib

correlation_matrix = time_by_location_hour_pd[["avg_trip_time_minutes", "avg_temp", "avg_rhum", "avg_speed_kmh", "avg_prcp", "avg_events", "demand_count","avg_distance", "avg_delay_minutes"]].corr()

# Configuro el tamaño del gráfico
plt.figure(figsize=(10, 8))

# Creo el mapa de calor
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, square=True, fmt=".2f",
            cbar_kws={"shrink": .8}, annot_kws={"size": 10})

# Configuración adicional del del mapa de calor
plt.title("Matriz de Correlación General a nivel de ruta", fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.tight_layout()

# Mostrar el gráfico
plt.show()

In [0]:
# Aquí calculo la desviacion estandar de akgunos atributos para evidenciar por qué las correlaciones son altas analizandolo de forma mas general
# y más bajas a nivel de ruta

# Sin coordenadas
time_by_location_hour_day.select(F.stddev("avg_speed_kmh"), F.stddev("avg_prcp")).show()

# Con coordenadas
time_by_location_hour.select(F.stddev("avg_speed_kmh"), F.stddev("avg_prcp")).show()


In [0]:
time_by_location_hour_pd

In [0]:
# Convierto la columna 'day_of_week' a tipo entero
time_by_location_hour_day_pd['day_of_week'] = time_by_location_hour_day_pd['day_of_week'].astype(int)

# Filtro los datos para días laborables y fines de semana (teniendo en cuenta desde el viernes al domingo, no incluido)
workdays = time_by_location_hour_day_pd[time_by_location_hour_day_pd['day_of_week'] <= 6]
weekends = time_by_location_hour_day_pd[time_by_location_hour_day_pd['day_of_week'] >= 7]

# Calculo las matrices de correlación para cada subconjunto
corr_workdays = workdays[["avg_trip_time_minutes", "avg_temp", "avg_rhum", "avg_speed_kmh", "avg_prcp", 
                          "avg_events", "avg_wspd","demand_count", "avg_distance"]].corr()

corr_weekends = weekends[["avg_trip_time_minutes", "avg_temp", "avg_rhum", "avg_speed_kmh", "avg_prcp", 
                          "avg_events", "avg_wspd","demand_count", "avg_distance"]].corr()

# Creo los subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Creo el mapa de calor para los días laborables
sns.heatmap(corr_workdays, annot=True, cmap="coolwarm", vmin=-1, vmax=1, square=True, fmt=".2f",
            cbar_kws={"shrink": .8}, annot_kws={"size": 10}, ax=ax1)
ax1.set_title("Correlación en Días Laborables", fontsize=14)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45)
ax1.set_yticklabels(ax1.get_yticklabels(), rotation=45)

# Creo el mapa de calor para los fines de semana
sns.heatmap(corr_weekends, annot=True, cmap="coolwarm", vmin=-1, vmax=1, square=True, fmt=".2f",
            cbar_kws={"shrink": .8}, annot_kws={"size": 10}, ax=ax2)
ax2.set_title("Correlación en Fines de Semana", fontsize=14)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45)
ax2.set_yticklabels(ax2.get_yticklabels(), rotation=45)

# Imprimo el gráfico en pantalla
plt.tight_layout()
plt.show()


In [0]:
# Creo una tabla pivote para la demanda
demand_pivot = time_by_location_hour_pd.pivot_table(
    index='day_of_week', 
    columns='hour', 
    values='demand_count', 
    aggfunc='sum'
)

# Creo una tabla pivote para los tiempos de viaje
trip_time_pivot = time_by_location_hour_pd.pivot_table(
    index='day_of_week', 
    columns='hour', 
    values='avg_trip_time_minutes', 
    aggfunc='mean'
)

# Creo una tabla pivote para la velocidad
speed_time_pivot = time_by_location_hour_pd.pivot_table(
    index='day_of_week', 
    columns='hour', 
    values='avg_speed_kmh', 
    aggfunc='mean'
)

# Creo el mapa de calor para la demanda
plt.figure(figsize=(12, 6))
sns.heatmap(demand_pivot, annot=False, fmt='.0f', cmap="YlGnBu", cbar_kws={'label': 'Demanda'}, linewidths=0.5)
plt.title('Mapa de Calor: Demanda vs Hora del Día y Día de la Semana')
plt.xlabel('Hora del Día')
plt.ylabel('Día de la Semana')
plt.show()

# Creo el mapa de calor para los tiempos de viaje
plt.figure(figsize=(12, 6))
sns.heatmap(trip_time_pivot, annot=False, fmt='.0f', cmap="coolwarm", cbar_kws={'label': 'Tiempo de Viaje (minutos)'}, linewidths=0.5)
plt.title('Mapa de Calor: Tiempo de Viaje Promedio vs Hora del Día y Día de la Semana')
plt.xlabel('Hora del Día')
plt.ylabel('Día de la Semana')
plt.show()

# Creo el mapa de calor para la velocidad
plt.figure(figsize=(12, 6))
sns.heatmap(speed_time_pivot, annot=False, fmt='.0f', cmap="coolwarm", cbar_kws={'label': 'Velocidad promedio (kmh)'}, linewidths=0.5)
plt.title('Mapa de Calor: Velocidad promedio vs Hora del Día y Día de la Semana')
plt.xlabel('Hora del Día')
plt.ylabel('Día de la Semana')
plt.show()

In [0]:
display(uber_weather_df)

In [0]:
uber_weather_df.columns

In [0]:
# Creo lista con los atributos que contienen la cantidad de eventos
event_columns = ['Athletic Race / Tour', 'BID Multi-Block', 'Bike the Block', 'Block Party', 
                 'Clean-Up', 'Farmers Market', 'Grid Request', 'Health Fair', 
                 'Open Culture', 'Open Street Partner Event', 'Parade', 
                 'Plaza Event', 'Plaza Partner Event', 'Press Conference', 
                 'Production Event', 'Religious Event', 'Sidewalk Sale', 
                 'Single Block Festival', 'Special Event', 'Sport - Adult', 
                 'Sport - Youth', 'Stationary Demonstration', 'Stickball', 
                 'Street Event', 'Street Festival', 'Theater Load in and Load Outs']

# Lista de las condiciones meteorológicas específicas
meteorological_columns = ['coco_1.0','coco_20.0', 'coco_15.0','coco_17.0',
                          'coco_9.0','coco_12.0','coco_18.0',
                          'coco_5.0', 'coco_4.0', 'coco_7.0',
                          'coco_2.0', 'coco_8.0','coco_3.0','coco_16.0','coco_13.0']

# Selecciono los atributos de interés
columns_of_interest = event_columns + meteorological_columns + ['total_trip_time_minutes_avg', 'speed_kmh_avg']

# Realizo la agregación de 'speed_kmh' y 'total_trip_time_minutes' por hora y día
agg_df = uber_weather_df.groupBy('hour_of_day', 'day_of_week').agg(
    avg('speed_kmh').alias('speed_kmh_avg'),
    avg('total_trip_time_minutes').alias('total_trip_time_minutes_avg')
)
# Incorporo el resto de atributos que necesito
agg_df = agg_df.join(
    uber_weather_df.select(
        'hour_of_day', 'day_of_week','Athletic Race / Tour', 'BID Multi-Block', 'Bike the Block',
        'Block Party', 'Clean-Up', 'Farmers Market', 'Grid Request', 'Health Fair',
        'Open Culture', 'Open Street Partner Event', 'Parade', 'Plaza Event', 
        'Plaza Partner Event', 'Press Conference', 'Production Event', 'Religious Event', 
        'Sidewalk Sale', 'Single Block Festival', 'Special Event', 'Sport - Adult', 
        'Sport - Youth', 'Stationary Demonstration', 'Stickball', 'Street Event', 
        'Street Festival', 'Theater Load in and Load Outs', 'nº events', 'Hour_int', 
        'time', 'temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 
        'pres', 'tsun', '`coco_1.0`', '`coco_20.0`', '`coco_15.0`', '`coco_17.0`', 
        '`coco_0.0`', '`coco_9.0`', '`coco_12.0`', '`coco_18.0`', '`coco_14.0`', 
        '`coco_19.0`', '`coco_5.0`', '`coco_4.0`', '`coco_7.0`', '`coco_2.0`', 
        '`coco_8.0`', '`coco_3.0`', '`coco_16.0`', '`coco_13.0`', 'total_trip_time_hours'
    ),
    on=['hour_of_day', 'day_of_week'],
    how='left'
)

# Convierto el dataframe de PySpark a pandas para realizar el cálculo de la correlación
df_pandas = agg_df.toPandas()

# Filtro columnas que están en columns_of_interest y que existen en el DataFrame
columns_of_interest = [col for col in columns_of_interest if col in df_pandas.columns]

# Elimino columnas con todos los valores nulos
df_pandas = df_pandas.dropna(axis=1, how='all')

# Elimino columnas con solo un valor único, pero preservando las columnas de interés
df_pandas = df_pandas.loc[:, df_pandas.nunique() > 1]

# Verifico que las columnas de interés están presentes después de la limpieza
columns_of_interest = [col for col in columns_of_interest if col in df_pandas.columns]

# Esto me avisa de si alguna columna de interés ha sido eliminada
if len(columns_of_interest) < len(event_columns + meteorological_columns + ['total_trip_time_minutes_avg', 'speed_kmh_avg']):
    missing_columns = set(event_columns + meteorological_columns + ['total_trip_time_minutes_avg', 'speed_kmh_avg']) - set(columns_of_interest)
    print(f"Advertencia: Algunas columnas de interés han sido eliminadas: {missing_columns}")

# Calculo la matriz de correlación
correlation_matrix = df_pandas[columns_of_interest].corr()

# Filtro la matriz de correlación para mostrar solo las columnas de 'total_trip_time_minutes' y 'speed_kmh' con el resto
correlation_matrix_filtered = correlation_matrix[['total_trip_time_minutes_avg', 'speed_kmh_avg']]

# Imprimo en àntalla la matriz de correlación con un mapa de calor
plt.figure(figsize=(10, 8))  
sns.heatmap(correlation_matrix_filtered, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, center=0, cbar_kws={'shrink': 0.75})
plt.title('Correlación entre `total_trip_time_minutes` y `speed_kmh` con el resto de las variables', fontsize=16)
plt.show()


In [0]:
correlation_matrix.columns

In [0]:
# Filtro las columnas que tienen alta correlación con 'speed_kmh' y 'total_trip_time_minutes'
correlation_with_speed = correlation_matrix['speed_kmh_avg'].sort_values(ascending=False)
correlation_with_trip_time = correlation_matrix['total_trip_time_minutes_avg'].sort_values(ascending=False)
correlation_with_trip_time.dropna(inplace=True)
correlation_with_speed.dropna(inplace=True)
# Imprimo los atributos más correlacionados
print("Atributos más correlacionados positivamente con 'speed_kmh':")
print(correlation_with_speed.head(10))  # Esto muestra los 10 atributos más correlacionados
print("Atributos más correlacionados negativamente con 'speed_kmh':")
print(correlation_with_speed.tail(10)) 
print("\nAtributos más correlacionados con 'total_trip_time_minutes':")
print(correlation_with_trip_time.head(10))
print("Atributos más correlacionados negativamente con 'total_trip_time_minutes':")
print(correlation_with_trip_time.tail(10)) 

In [0]:
# Creo lista con los atributos que contienen la cantidad de eventos
event_columns = ['Athletic Race / Tour', 'BID Multi-Block', 'Bike the Block', 'Block Party', 
                 'Clean-Up', 'Farmers Market', 'Grid Request', 'Health Fair', 
                 'Open Culture', 'Open Street Partner Event', 'Parade', 
                 'Plaza Event', 'Plaza Partner Event', 'Press Conference', 
                 'Production Event', 'Religious Event', 'Sidewalk Sale', 'Single Block Festival', 
                 'Special Event', 'Sport - Adult', 'Sport - Youth', 'Stationary Demonstration', 
                 'Stickball', 'Street Event', 'Street Festival', 'Theater Load in and Load Outs']

# Lista de las condiciones meteorológicas específicas
meteorological_columns = ['coco_1.0','coco_20.0', 'coco_15.0','coco_17.0',
                          'coco_9.0','coco_12.0','coco_18.0','coco_5.0', 
                          'coco_4.0', 'coco_7.0','coco_2.0','coco_8.0','coco_3.0','coco_16.0','coco_13.0']

# Selecciono las columnas de interés
columns_of_interest = event_columns + meteorological_columns + ['total_trip_time_minutes', 'speed_kmh', 'trip_kilometers']

# Convierto el dataframe de PySpark a pandas para realizar el cálculo de la correlación
df_pandas = uber_weather_df.toPandas()

# Filtro columnas que están en columns_of_interest y que existen en el DataFrame
columns_of_interest = [col for col in columns_of_interest if col in df_pandas.columns]

# Elimino columnas con todos los valores nulos
df_pandas = df_pandas.dropna(axis=1, how='all')

# Elimino columnas con solo un valor único, pero preservando las columnas de interés
df_pandas = df_pandas.loc[:, df_pandas.nunique() > 1]

# Verifico que las columnas de interés están presentes después de la limpieza
columns_of_interest = [col for col in columns_of_interest if col in df_pandas.columns]

# Esto me avisa de si alguna columna de interés ha sido eliminada
if len(columns_of_interest) < len(event_columns + meteorological_columns + ['total_trip_time_minutes', 'speed_kmh', 'trip_kilometers']):
    missing_columns = set(event_columns + meteorological_columns + ['total_trip_time_minutes', 'speed_kmh', 'trip_kilometers']) - set(columns_of_interest)
    print(f"Advertencia: Algunas columnas de interés han sido eliminadas: {missing_columns}")

# Divido los trayectos en cortos y largos según los kilómetros del trayecto
median_trip_km = df_pandas['trip_kilometers'].median()

# Filtro los trayectos cortos (menos que la mediana de kilómetros)
df_short_trips = df_pandas[df_pandas['trip_kilometers'] < median_trip_km]

# Filtro los trayectos largos (mayores o iguales a la mediana de kilómetros)
df_long_trips = df_pandas[df_pandas['trip_kilometers'] >= median_trip_km]

# Calculo la matriz de correlación para trayectos cortos
correlation_matrix_short = df_short_trips[columns_of_interest].corr()

# Filtro la matriz de correlación para mostrar solo las columnas de 'total_trip_time_minutes' y 'speed_kmh'
correlation_matrix_short_filtered = correlation_matrix_short[['total_trip_time_minutes', 'speed_kmh']]

# Calculo la matriz de correlación para trayectos largos
correlation_matrix_long = df_long_trips[columns_of_interest].corr()

# Filtro la matriz de correlación para mostrar solo las columnas de 'total_trip_time_minutes' y 'speed_kmh'
correlation_matrix_long_filtered = correlation_matrix_long[['total_trip_time_minutes', 'speed_kmh']]

# Imprimo la matriz de correlación para trayectos cortos
plt.figure(figsize=(10, 8))  # Tamaño de la figura adecuado
sns.heatmap(correlation_matrix_short_filtered, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, center=0, cbar_kws={'shrink': 0.75})
plt.title('Correlación entre `total_trip_time_minutes` y `speed_kmh` (Trayectos Cortos)', fontsize=16)
plt.show()

# Imprimo la matriz de correlación para trayectos largos
plt.figure(figsize=(10, 8))  # Tamaño de la figura adecuado
sns.heatmap(correlation_matrix_long_filtered, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, center=0, cbar_kws={'shrink': 0.75})
plt.title('Correlación entre `total_trip_time_minutes` y `speed_kmh` (Trayectos Largos)', fontsize=16)
plt.show()



In [0]:
df_pandas.columns

In [0]:
# Creo lista con los atributos que contienen la cantidad de eventos
event_columns = ['Athletic Race / Tour', 'BID Multi-Block', 'Bike the Block', 'Block Party', 
                 'Clean-Up', 'Farmers Market', 'Grid Request', 'Health Fair', 
                 'Open Culture', 'Open Street Partner Event', 'Parade', 
                 'Plaza Event', 'Plaza Partner Event', 'Press Conference', 
                 'Production Event', 'Religious Event', 'Sidewalk Sale', 
                 'Single Block Festival', 'Special Event', 'Sport - Adult', 
                 'Sport - Youth', 'Stationary Demonstration', 'Stickball', 
                 'Street Event', 'Street Festival', 'Theater Load in and Load Outs']

# Lista de las condiciones meteorológicas específicas
meteorological_columns = ['coco_1.0','coco_20.0', 'coco_15.0','coco_17.0',
                          'coco_9.0','coco_12.0','coco_18.0',
                          'coco_5.0', 'coco_4.0', 'coco_7.0',
                          'coco_2.0', 'coco_8.0','coco_3.0','coco_16.0','coco_13.0']

# Selecciono las columnas de interés
columns_of_interest = event_columns + meteorological_columns + ['total_trip_time_minutes_avg', 'speed_kmh_avg']

# Realizo la agregación de 'speed_kmh' y 'total_trip_time_minutes' por hora y día
agg_df = uber_weather_df.groupBy('hour_of_day', 'day_of_week').agg(
    avg('speed_kmh').alias('speed_kmh_avg'),
    avg('total_trip_time_minutes').alias('total_trip_time_minutes_avg')
)

# Incorporo las demás columnas que necesito
agg_df = agg_df.join(
    uber_weather_df.select(
        'hour_of_day', 'day_of_week','Athletic Race / Tour', 'BID Multi-Block', 'Bike the Block',
        'Block Party', 'Clean-Up', 'Farmers Market', 'Grid Request', 'Health Fair',
        'Open Culture', 'Open Street Partner Event', 'Parade', 'Plaza Event', 
        'Plaza Partner Event', 'Press Conference', 'Production Event', 'Religious Event', 
        'Sidewalk Sale', 'Single Block Festival', 'Special Event', 'Sport - Adult', 
        'Sport - Youth', 'Stationary Demonstration', 'Stickball', 'Street Event', 
        'Street Festival', 'Theater Load in and Load Outs', 'nº events', 'Hour_int', 
        'time', 'temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 
        'pres', 'tsun', '`coco_1.0`', '`coco_20.0`', '`coco_15.0`', '`coco_17.0`', 
        '`coco_0.0`', '`coco_9.0`', '`coco_12.0`', '`coco_18.0`', '`coco_14.0`', 
        '`coco_19.0`', '`coco_5.0`', '`coco_4.0`', '`coco_7.0`', '`coco_2.0`', 
        '`coco_8.0`', '`coco_3.0`', '`coco_16.0`', '`coco_13.0`', 'total_trip_time_hours'
    ),
    on=['hour_of_day', 'day_of_week'],
    how='left'
)

# Convierto el dataframe de PySpark a pandas para realizar el cálculo de la correlación
df_pandas = agg_df.toPandas()

# Filtro columnas que están en columns_of_interest y que existen en el DataFrame
columns_of_interest = [col for col in columns_of_interest if col in df_pandas.columns]

# Elimino columnas con todos los valores nulos
df_pandas = df_pandas.dropna(axis=1, how='all')

# Elimino columnas con solo un valor único, pero preservando las columnas de interés
df_pandas = df_pandas.loc[:, df_pandas.nunique() > 1]

# Verifico que las columnas de interés están presentes después de la limpieza
columns_of_interest = [col for col in columns_of_interest if col in df_pandas.columns]

# Esto me avisa de si alguna columna de interés ha sido eliminada
if len(columns_of_interest) < len(event_columns + meteorological_columns + ['total_trip_time_minutes_avg', 'speed_kmh_avg']):
    missing_columns = set(event_columns + meteorological_columns + ['total_trip_time_minutes_avg', 'speed_kmh_avg']) - set(columns_of_interest)
    print(f"Advertencia: Algunas columnas de interés han sido eliminadas: {missing_columns}")

# Defino los momentos del día según la columna 'hour_of_day'
def get_moment_of_day(hour):
    if 0 <= hour <= 5:
        return 'Madrugada'
    elif 6 <= hour <= 11:
        return 'Mañana'
    elif 12 <= hour <= 17:
        return 'Tarde'
    elif 18 <= hour <= 23:
        return 'Noche'
    else:
        return 'Desconocido'  # En caso de valores fuera de rango

# Creo la columna 'moment_of_day' en el df
df_pandas['moment_of_day'] = df_pandas['hour_of_day'].apply(get_moment_of_day)

# Creo listas de momentos del día
moments = ['Madrugada', 'Mañana', 'Tarde', 'Noche']

# Creo la figura para subplots 2x2
fig, axes = plt.subplots(2, 2, figsize=(15, 12))  # Tamaño adecuado para los subplots

# Ajusto el espacio entre subplots (mejora la visualización en la interfaz)
plt.subplots_adjust(hspace=0.3, wspace=0.3)

# Creo los subplots para visualizar correlaciones por momento del día
for i, moment in enumerate(moments):
    ax = axes[i // 2, i % 2]  # Determina la posición del subplot (2x2)

    # Filtro por momento del día
    df_filtered = df_pandas[df_pandas['moment_of_day'] == moment]

    if df_filtered.empty:
        print(f"No hay datos para el momento '{moment}'.")
        continue

    # Calculo la matriz de correlación solo con las columnas de interés existentes
    correlation_matrix = df_filtered[columns_of_interest].corr()
    correlation_filtered = correlation_matrix[['total_trip_time_minutes_avg', 'speed_kmh_avg']]

    # Visualizo la matriz de correlación con un heatmap
    sns.heatmap(correlation_filtered, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, center=0, cbar_kws={'shrink': 0.75}, ax=ax)
    ax.set_title(f'Correlación en {moment}', fontsize=14)

plt.show()


In [0]:
time_by_location_hour_pd

In [0]:
# Encuentro las rutas más populares por demanda
top_routes = (
    time_by_location_hour_pd.groupby(
        ["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"]
    )["demand_count"]
    .sum()
    .reset_index()
    .sort_values("demand_count", ascending=False)
    .head(10)  # Top 10 rutas
)

print("Top 10 rutas con mayor demanda:")
display(top_routes)

In [0]:
print("Número de rutas en el top_routes:", len(top_routes))


In [0]:
# Creo un mapa centrado en la ubicación promedio de las rutas más populares
center_lat = top_routes["pickup_latitude"].mean()
center_lon = top_routes["pickup_longitude"].mean()
map_routes = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Creo un clúster para los marcadores
marker_cluster = MarkerCluster().add_to(map_routes)

# Lista de colores para las rutas
colors = ['blue', 'green', 'orange', 'red', 'purple', 'darkred', 'lightred', 'beige', 'darkblue', 'cadetblue']

# Añado solo las 10 rutas más populares al mapa
for i, (_, row) in enumerate(top_routes.iterrows()):
    origin = [row["pickup_latitude"], row["pickup_longitude"]]
    destination = [row["dropoff_latitude"], row["dropoff_longitude"]]

    # Añado marcadores para el origen y destino
    folium.Marker(
        origin,
        icon=folium.Icon(color='green', icon='arrow-up', prefix='fa'),
        tooltip="Origen"
    ).add_to(marker_cluster)

    folium.Marker(
        destination,
        icon=folium.Icon(color='red', icon='arrow-down', prefix='fa'),
        tooltip="Destino"
    ).add_to(marker_cluster)

    # Añado línea entre el origen y el destino para visualizar la ruta
    folium.PolyLine(
        [origin, destination],
        color=colors[i % len(colors)],  # Usar un color de la lista
        weight=5,  # Grosor de la línea
        opacity=0.7,  # Opacidad de la línea
        tooltip=f"Demand Count: {row['demand_count']}"
    ).add_to(map_routes)

# Imprimo el mapa en pantalla
display(map_routes)

In [0]:
# Rutas con alta demanda (top 10%)
high_demand_routes = time_by_location_hour_pd[
    time_by_location_hour_pd["demand_count"] >= time_by_location_hour_pd["demand_count"].quantile(0.9)
]

# Rutas con baja demanda (bottom 10%)
low_demand_routes = time_by_location_hour_pd[
    time_by_location_hour_pd["demand_count"] <= time_by_location_hour_pd["demand_count"].quantile(0.1)
]

print("Rutas con alta demanda (top 10%):")
display(high_demand_routes)

print("Rutas con baja demanda (bottom 10%):")
display(low_demand_routes)


In [0]:
# Calculo la distancia por medio de Función Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c  # Distancia en km

# Calculo la distancia entre pares de ubicaciones
time_by_location_hour_pd["avg_distance"] = time_by_location_hour_pd.apply(
    lambda row: haversine(
        row["pickup_latitude"], row["pickup_longitude"],
        row["dropoff_latitude"], row["dropoff_longitude"]
    ), axis=1
)

# Agrupo los datos en función de la distancia y analizo la correlación con las variables meteorológicas
short_trips = time_by_location_hour_pd[time_by_location_hour_pd["avg_distance"] <= 5]
long_trips = time_by_location_hour_pd[time_by_location_hour_pd["avg_distance"] > 5]

print("Correlación para trayectos cortos (<=5 km):")
print(short_trips[["avg_trip_time_minutes", "avg_temp", "avg_rhum", "avg_speed_kmh", "avg_prcp", "avg_events", "demand_count"]].corr())

print("Correlación para trayectos largos (>5 km):")
print(long_trips[["avg_trip_time_minutes", "avg_temp", "avg_rhum", "avg_speed_kmh", "avg_prcp", "avg_events", "demand_count"]].corr())


In [0]:
demand_by_location_date = uber_weather_df.groupBy(
    F.col("pickup_latitude").alias("latitude"),
    F.col("pickup_longitude").alias("longitude"),
    F.to_date("request_datetime").alias("date")
).count().alias("demand_count")

# Convierto a Pandas para pasar a Folium
demand_by_location_date_pd = demand_by_location_date.toPandas()

In [0]:
demand_by_location_date_pd

In [0]:
# Agrupo por las columnas deseadas y calcular la cantidad total de solicitudes y el retraso promedio
grouped_data = uber_weather_df.groupBy(
    'pickup_latitude',
    'pickup_longitude',
    'dropoff_latitude',
    'dropoff_longitude',
    'hour_of_day',
    'day_of_week'
).agg(
    F.count('pickup_location_id').alias('total_requests'),  # Total de solicitudes
    F.avg('delay_minutes').alias('avg_delay')  # Retraso promedio
)

# Imprimo los resultados
grouped_data.show()


In [0]:

# Creo un mapa centrado en Nueva York
map_ny = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Filtro y agrupo los datos en Spark para minimizar tamaño
grouped_data_filtered = (
    grouped_data
    .filter(
        (grouped_data.pickup_latitude.isNotNull()) &
        (grouped_data.pickup_longitude.isNotNull()) &
        (grouped_data.total_requests.isNotNull())
    )
    .groupBy("pickup_latitude", "pickup_longitude")
    .sum("total_requests")
    .withColumnRenamed("sum(total_requests)", "total_requests")
)

# Convierto a pandas solo las columnas necesarias para el mapa
grouped_data_pd = grouped_data_filtered.select("pickup_latitude", "pickup_longitude", "total_requests").toPandas()
grouped_data_pd = grouped_data_pd.dropna()
# Creo los datos para el mapa de calor
heat_data = [
    [row['pickup_latitude'], row['pickup_longitude'], row['total_requests']] 
    for index, row in grouped_data_pd.iterrows()
]
HeatMap(heat_data, radius=15).add_to(map_ny)

# Visualizo el mapa en pantalla en Jupyter
display(map_ny) 



In [0]:
# Creo un mapa centrado en Nueva York para los retrasos
map_delays = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Filtro y agrupo los datos en Spark para reducir el tamaño
grouped_data_filtered = (
    grouped_data
    .filter(
        (grouped_data.pickup_latitude.isNotNull()) &
        (grouped_data.pickup_longitude.isNotNull()) &
        (grouped_data.avg_delay.isNotNull())
    )
    .groupBy("pickup_latitude", "pickup_longitude")
    .avg("avg_delay")
    .withColumnRenamed("avg(avg_delay)", "avg_delay")
)

# Convierto a pandas solo las columnas necesarias para el mapa
grouped_data_pd = grouped_data_filtered.select("pickup_latitude", "pickup_longitude", "avg_delay").toPandas()

grouped_data_pd= grouped_data_pd.dropna()
# Creo los datos para el mapa de calor de retrasos
delay_data = [
    [row['pickup_latitude'], row['pickup_longitude'], row['avg_delay']]
    for index, row in grouped_data_pd.iterrows()
]
HeatMap(delay_data, radius=15).add_to(map_delays)

# Imprimo el mapa en pantalla
display(map_delays)

In [0]:
# Selecciono las características relevantes para el clustering
features = time_by_location_hour_pd[[
    "demand_count", "avg_prcp",
    "avg_speed_kmh", "avg_trip_time_minutes", "avg_events", "avg_distance"
]]

# Elimino las filas con valores NaN en las características relevantes
features_clean = features.dropna()

# Normalizo las características
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_clean)

# Aplico KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(scaled_features)

# Añado la columna con el cluster pertinente
features_clean['cluster'] = clusters

# Muestra solo la media de las características para cada clúster
cluster_means = features_clean.groupby('cluster').mean()

# Muestra el resumen de las medias por clúster
print("Medias de las características por clúster:")
display(cluster_means)
